In [36]:
import numpy as np
from scipy.optimize import minimize
from scipy.io import loadmat
from math import sqrt

In [37]:
def initializeWeights(n_in, n_out):
    """
    # initializeWeights return the random weights for Neural Network given the
    # number of node in the input layer and output layer

    # Input:
    # n_in: number of nodes of the input layer
    # n_out: number of nodes of the output layer
       
    # Output: 
    # W: matrix of random initial weights with size (n_out x (n_in + 1))"""

    epsilon = sqrt(6) / sqrt(n_in + n_out + 1)
    W = (np.random.rand(n_out, n_in + 1) * 2 * epsilon) - epsilon
    return W

In [38]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [39]:
def preprocess():
    """ Input:
     Although this function doesn't have any input, you are required to load
     the MNIST data set from file 'mnist_all.mat'.

     Output:
     train_data: matrix of training set. Each row of train_data contains 
       feature vector of a image
     train_label: vector of label corresponding to each image in the training
       set
     validation_data: matrix of training set. Each row of validation_data 
       contains feature vector of a image
     validation_label: vector of label corresponding to each image in the 
       training set
     test_data: matrix of training set. Each row of test_data contains 
       feature vector of a image
     test_label: vector of label corresponding to each image in the testing
       set

     Some suggestions for preprocessing step:
     - feature selection"""

    mat = loadmat('mnist_all.mat')  


def preprocess():
    """
    Load the MNIST dataset, split it into training, validation, and test sets, 
    and normalize the data.
    
    Returns:
        train_data: Training data matrix
        train_label: Labels for the training data
        validation_data: Validation data matrix
        validation_label: Labels for the validation data
        test_data: Test data matrix
        test_label: Labels for the test data
    """
    
    # Load the MNIST data
    mat = loadmat('mnist_all.mat')  # MNIST dataset is loaded as a dictionary

    # Initializing list
    train_data_list, train_label_list = [], []
    validation_data_list, validation_label_list = [], []
    test_data_list, test_label_list = [], []

    # Loop for each digits
    for digit in range(10):
        # Load training and test data for the current digit
        train_key = f'train{digit}'
        test_key = f'test{digit}'
        train_images = mat[train_key]
        test_images = mat[test_key]
        np.random.shuffle(train_images)
        split_index = 1000 
        
        train_data_list.append(train_images[split_index:])
        train_label_list.append(np.full(train_images.shape[0] - split_index, digit))
        
        validation_data_list.append(train_images[:split_index])
        validation_label_list.append(np.full(split_index, digit))
        
        test_data_list.append(test_images)
        test_label_list.append(np.full(test_images.shape[0], digit))
    
    # Here we are concating all
    train_data = np.vstack(train_data_list)
    train_label = np.concatenate(train_label_list)
    validation_data = np.vstack(validation_data_list)
    validation_label = np.concatenate(validation_label_list)
    test_data = np.vstack(test_data_list)
    test_label = np.concatenate(test_label_list)
    
    # Normalizing 
    train_data = train_data.astype(np.float32) / 255.0
    validation_data = validation_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    
    train_perm = np.random.permutation(train_data.shape[0])
    train_data, train_label = train_data[train_perm], train_label[train_perm]
    
    validation_perm = np.random.permutation(validation_data.shape[0])
    validation_data, validation_label = validation_data[validation_perm], validation_label[validation_perm]
    
    print("Preprocessing done!")
    
    return train_data, train_label, validation_data, validation_label, test_data, test_label


In [40]:
def nnObjFunction(params, *args):
    n_input, n_hidden, n_class, training_data, training_label, lambdaval = args
    
    # Reshape 'params' to get weight matrices w1 and w2
    w1 = params[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
    w2 = params[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

    # Number of training samples
    num_samples = training_data.shape[0]
    
    # Forward propagation
    # Add bias to the input layer
    training_data = np.hstack((training_data, np.ones((num_samples, 1))))
    
    # Input to the hidden layer
    hidden_input = np.dot(training_data, w1.T)
    hidden_output = sigmoid(hidden_input)
    
    # Add bias to the hidden layer
    hidden_output = np.hstack((hidden_output, np.ones((hidden_output.shape[0], 1))))
    
    # Input to the output layer
    output_input = np.dot(hidden_output, w2.T)
    output = sigmoid(output_input)
    
    # One-hot encoding for labels
    y = np.zeros((num_samples, n_class))
    y[np.arange(num_samples), training_label.astype(int)] = 1

    # Calculate error (cross-entropy or log-likelihood error with regularization)
    log_error = y * np.log(output) + (1 - y) * np.log(1 - output)
    obj_val = -np.sum(log_error) / num_samples

    # Regularization term for error
    obj_val += (lambdaval / (2 * num_samples)) * (np.sum(np.square(w1)) + np.sum(np.square(w2)))
    
    # Backpropagation
    delta_output = output - y
    grad_w2 = np.dot(delta_output.T, hidden_output) / num_samples + (lambdaval / num_samples) * w2

    # Error propagated back to hidden layer
    delta_hidden = np.dot(delta_output, w2) * hidden_output * (1 - hidden_output)
    delta_hidden = delta_hidden[:, :-1]  # Remove bias term from backprop error
    grad_w1 = np.dot(delta_hidden.T, training_data) / num_samples + (lambdaval / num_samples) * w1

    # Flatten gradients
    obj_grad = np.concatenate((grad_w1.flatten(), grad_w2.flatten()), 0)

    return obj_val, obj_grad


In [41]:
def nnPredict(w1, w2, data):
    data = np.hstack((data, np.ones((data.shape[0], 1))))
    hidden_input = np.dot(data, w1.T)
    hidden_output = sigmoid(hidden_input)
    hidden_output = np.hstack((hidden_output, np.ones((hidden_output.shape[0], 1))))
    output_input = np.dot(hidden_output, w2.T)
    output = sigmoid(output_input)

    labels = np.argmax(output, axis=1)
    return labels


In [42]:
"""**************Neural Network Script Starts here********************************"""

# Preprocess the data
train_data, train_label, validation_data, validation_label, test_data, test_label = preprocess()

# Set hyperparameters
# Number of nodes in input layer (not including bias unit)
n_input = train_data.shape[1]

# Number of nodes in hidden layer (not including bias unit)
n_hidden = 50  # You can adjust this for experimentation

# Number of nodes in output layer
n_class = 10

# Regularization hyperparameter (adjustable)
lambdaval = 6  # You can try values like 0.1, 1.0, etc., for tuning

# Initialize weights for both layers with random values
initial_w1 = initializeWeights(n_input, n_hidden)
initial_w2 = initializeWeights(n_hidden, n_class)

# Unroll 2D weight matrices into a single vector for optimization
initialWeights = np.concatenate((initial_w1.flatten(), initial_w2.flatten()), 0)

# Arguments to pass to the objective function
args = (n_input, n_hidden, n_class, train_data, train_label, lambdaval)

# Set optimization options
opts = {'maxiter': 50}  # Maximum iterations, adjustable if more training is needed

# Train Neural Network using the minimize function with Conjugate Gradient (CG) method
nn_params = minimize(nnObjFunction, initialWeights, jac=True, args=args, method='CG', options=opts)

# Alternative (optional) usage of fmin_cg
# nn_params, cost = fmin_cg(nnObjFunctionVal, initialWeights, fprime=nnObjGradient, args=args, maxiter=50)

# Reshape nn_params back into w1 and w2 matrices after training
w1 = nn_params.x[0:n_hidden * (n_input + 1)].reshape((n_hidden, (n_input + 1)))
w2 = nn_params.x[(n_hidden * (n_input + 1)):].reshape((n_class, (n_hidden + 1)))

# Test the neural network's performance on training, validation, and test sets

# Training accuracy
predicted_label = nnPredict(w1, w2, train_data)
print('\n Training set Accuracy: ' + str(100 * np.mean((predicted_label == train_label).astype(float))) + '%')

# Validation accuracy
predicted_label = nnPredict(w1, w2, validation_data)
print('\n Validation set Accuracy: ' + str(100 * np.mean((predicted_label == validation_label).astype(float))) + '%')

# Test accuracy
predicted_label = nnPredict(w1, w2, test_data)
print('\n Test set Accuracy: ' + str(100 * np.mean((predicted_label == test_label).astype(float))) + '%')


Preprocessing done!

 Training set Accuracy: 95.256%

 Validation set Accuracy: 94.88%

 Test set Accuracy: 94.94%
